In [54]:
import sqlalchemy
# import pymysql
import os
import pandas as pd

import sys
sys.path.append('../')
import private_info.db_info as dbi

In [55]:
engine = sqlalchemy.create_engine(f'mysql+pymysql://{dbi.username}:{dbi.password}@{dbi.hostname}:{dbi.port}/{dbi.database}')

In [56]:
db_sqla_conn = engine.connect()

In [57]:
print (db_sqla_conn)

In [58]:
files_list = []

for root, dir, files in os.walk('data'):
    for name in files:
        files_list.append(os.path.join(root,name))

In [59]:
print (files_list)

['data/asset/IWM.csv', 'data/asset/SPY.csv', 'data/asset/QQQ.csv', 'data/gen/fvx.csv', 'data/gen/fed_funds.csv', 'data/gen/tnx.csv', 'data/gen/vix.csv']


In [60]:
for path in files_list:
    
    df = pd.read_csv (path)
    path_parts = path.split('/')
    t_name = path_parts[2][:-4].lower()
    
    print (t_name)
    
    df.to_sql(t_name, engine, if_exists='fail')
    
    engine.execute(f"ALTER TABLE {t_name} DROP COLUMN `Unnamed: 0`;")
    
    engine.execute(f"ALTER TABLE {t_name} CHANGE COLUMN `index` `index` BIGINT NULL AUTO_INCREMENT;")
    
    results = engine.execute(f"SELECT Date FROM {t_name} ORDER BY Date ASC LIMIT 1;")
    results = results.fetchall()[0][0]
    
    engine.execute(f"UPDATE {t_name} SET `index` = 0 WHERE Date = \'{results}\';")
    
    engine.execute(f"ALTER TABLE {t_name} ADD PRIMARY KEY (`index`);")
    
    engine.execute( f"ALTER TABLE {t_name} AUTO_INCREMENT = 0;" )

    

iwm


Exception during reset or similar
Traceback (most recent call last):
  File "/home/sdk/RT_ETF_predictor/venv/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 739, in _finalize_fairy
    fairy._reset(pool)
  File "/home/sdk/RT_ETF_predictor/venv/lib/python3.9/site-packages/sqlalchemy/pool/base.py", line 988, in _reset
    pool._dialect.do_rollback(self)
  File "/home/sdk/RT_ETF_predictor/venv/lib/python3.9/site-packages/sqlalchemy/engine/default.py", line 682, in do_rollback
    dbapi_connection.rollback()
  File "/home/sdk/RT_ETF_predictor/venv/lib/python3.9/site-packages/pymysql/connections.py", line 480, in rollback
    self._read_ok_packet()
  File "/home/sdk/RT_ETF_predictor/venv/lib/python3.9/site-packages/pymysql/connections.py", line 443, in _read_ok_packet
    pkt = self._read_packet()
  File "/home/sdk/RT_ETF_predictor/venv/lib/python3.9/site-packages/pymysql/connections.py", line 692, in _read_packet
    packet_header = self._read_bytes(4)
  File "/home/sdk/RT_ETF_p

2000-06-01
spy
2000-06-01
qqq
2000-06-01
fvx
2000-06-01
fed_funds
2000-06-01
tnx
2000-06-01
vix
2000-06-01
